<a href="https://colab.research.google.com/github/SadikHasanKC/Flight_price_DL_ANN/blob/main/flight_predictor_DL_ANN_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group members: Frederik Kaminski Klitte, Patrick Riber Sørensen, Emmanouil Tzevelekos, Sadik Hasan Khan Chowdhury

Flight Price DL ANN

In [ ]:
!pip install scikit-learn
!pip install gradio
!pip install shap
!pip install imblearn
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.7 MB/

In [ ]:
# Importing the necessary libraries


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm_notebook
from imblearn.over_sampling import SMOTE
import torch
from sklearn.preprocessing import OneHotEncoder
import itertools
from sklearn.model_selection import train_test_split




In [297]:
# Reading in the dataset from the csv file

df = pd.read_csv('https://raw.githubusercontent.com/SadikHasanKC/Flight_price_DL_ANN/main/Clean_Dataset.csv', index_col=0)

In [298]:
df.shape

(300153, 11)

In [244]:
data = df.iloc[:20000]

In [245]:
data.shape

(20000, 11)

In [246]:
data.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [247]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [248]:
data.describe()

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,9.744345,25.849950,6125.027950
std,6.711598,13.553019,3647.292391
min,2.000000,1.000000,2281.000000
25%,4.920000,14.000000,3855.000000
50%,8.170000,26.000000,4896.000000
75%,12.580000,38.000000,7424.000000
max,36.920000,49.000000,31917.000000


In [249]:
data.isnull().sum()

airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [250]:
y= data['price']

In [251]:
# Assuming your DataFrame is named 'data'
columns_to_select = ['airline', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left']
x = data[columns_to_select]


In [252]:
x

,airline,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,SpiceJet,Evening,zero,Night,Mumbai,Economy,2.17,1
1,SpiceJet,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1
2,AirAsia,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1
3,Vistara,Morning,zero,Afternoon,Mumbai,Economy,2.25,1
4,Vistara,Morning,zero,Morning,Mumbai,Economy,2.33,1
...,...,...,...,...,...,...,...,...
19995,Vistara,Morning,one,Evening,Bangalore,Economy,6.08,49
19996,Vistara,Afternoon,one,Evening,Bangalore,Economy,6.42,49
19997,Vistara,Afternoon,one,Night,Bangalore,Economy,6.58,49
19998,Vistara,Afternoon,one,Night,Bangalore,Economy,7.33,49


In [253]:
y

0        5953
1        5953
2        5956
3        5955
4        5955
         ... 
19995    4496
19996    4496
19997    4496
19998    4496
19999    4496
Name: price, Length: 20000, dtype: int64

In [254]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming 'x' is your DataFrame
columns_to_encode = ['airline', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

# Extract the columns to be one-hot encoded
data_to_encode = x[columns_to_encode]

# Create an instance of OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(data_to_encode)

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

# Concatenate the original DataFrame with the encoded DataFrame
x_encoded = pd.concat([x, encoded_df], axis=1)

# Drop the original columns that have been encoded
x_encoded = x_encoded.drop(columns=columns_to_encode)

# Display the resulting DataFrame
print(x_encoded)


       duration  days_left  airline_Air_India  airline_GO_FIRST  \
0          2.17          1                0.0               0.0   
1          2.33          1                0.0               0.0   
2          2.17          1                0.0               0.0   
3          2.25          1                0.0               0.0   
4          2.33          1                0.0               0.0   
...         ...        ...                ...               ...   
19995      6.08         49                0.0               0.0   
19996      6.42         49                0.0               0.0   
19997      6.58         49                0.0               0.0   
19998      7.33         49                0.0               0.0   
19999      7.42         49                0.0               0.0   

       airline_Indigo  airline_SpiceJet  airline_Vistara  \
0                 0.0               1.0              0.0   
1                 0.0               1.0              0.0   
2              

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [255]:
from sklearn.model_selection import train_test_split

# Assuming 'y' is your target variable

# Drop the target variable from the features
X = x_encoded  # Replace 'price' with the actual column name for your target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (16000, 20)
X_test shape: (4000, 20)
y_train shape: (16000,)
y_test shape: (4000,)


In [256]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [257]:
# Convert y_train and y_test to NumPy arrays
y_train_array = y_train.to_numpy()
y_test_array = y_test.to_numpy()

# Initialize StandardScaler for y
scaler_y = StandardScaler()

# Fit and transform y_train
y_train = scaler_y.fit_transform(y_train_array.reshape(-1, 1))

# Transform y_test
y_test = scaler_y.transform(y_test_array.reshape(-1, 1))

In [258]:
X_train

array([[-0.39676776,  0.30690846, -0.53504863, ..., -0.47949739,
         1.48604902,  1.00388254],
       [ 0.08612414, -0.42919338,  1.86898901, ...,  2.08551708,
        -0.67292531,  1.00388254],
       [-0.13674904,  1.33745103, -0.53504863, ..., -0.47949739,
        -0.67292531,  1.00388254],
       ...,
       [-0.06245798,  0.15968809, -0.53504863, ..., -0.47949739,
        -0.67292531,  1.00388254],
       [ 0.39666078, -1.45973595, -0.53504863, ...,  2.08551708,
        -0.67292531,  1.00388254],
       [ 0.18567417,  0.30690846, -0.53504863, ...,  2.08551708,
        -0.67292531, -0.99613248]])

In [259]:
y_train

array([[-0.22462512],
       [-0.16906323],
       [ 0.87946126],
       ...,
       [-0.77281749],
       [ 1.65320192],
       [-0.82865445]])

In [260]:
# Convert X_train to PyTorch tensor
tensor_data_X = torch.tensor(X_train, dtype=torch.float32)

# Convert y_train to PyTorch tensor
tensor_data_Y = torch.tensor(y_train, dtype=torch.float32)


In [261]:
tensor_data_X[0]

tensor([-0.3968,  0.3069, -0.5350,  2.3260, -0.4655, -0.2428, -0.6252, -0.5056,
        -0.5807, -0.0986, -0.5418, -0.3390, -0.1670, -0.5133, -0.2261, -0.5506,
        -0.2646, -0.4795,  1.4860,  1.0039])

In [262]:
tensor_data_Y[0]

tensor([-0.2246])

In [263]:
tensor_data_X.reshape(-1, 1).shape

torch.Size([320000, 1])

Starting our initial training model with these hyperparameters
epochs = 3
learning_rate = 2
input_size = 20
Neurons = 50
activation_input = 50
activation function = relu
output = 1

In [278]:
import torch
from torch import nn, optim
from tqdm import tqdm_notebook
# Initializing Hyperparameters
epochs = 3
learning_rate = 2


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Sequential(torch.nn.Linear(20,50),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(50,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-278-89c1555ae612>:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 1.5597
Epoch 3, Sample 11007: Loss: 2.6384
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.0864
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.1246
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 1.4884
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 2.7925
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

# 2nd Iteration with different settings for hyperparameters

For second trial we chose these settings
epochs = 3
learning_rate = .01
input_size = 20
Neurons = 50
activation_input = 50
activation function = relu
output = 1

In [269]:
epochs = 3
learning_rate = .01


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.Linear(20,50),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(50,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net2[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-269-19b664e51985>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 0.1425
Epoch 3, Sample 11007: Loss: 0.2293
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.0864
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 1.2137
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.4425
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 0.1357
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 0.2539
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

For third trial we chose these settings
epochs = 3
learning_rate = .1
input_size = 20
Neurons = 50
activation_input = 50
activation function = relu
output = 1

In [279]:
epochs = 3
learning_rate = .1


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net3 = torch.nn.Sequential(torch.nn.Linear(20,50),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(50,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net3.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net3.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net3[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-279-c633475578be>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 1.5597
Epoch 3, Sample 11007: Loss: 2.6384
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.0864
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.1246
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 1.4884
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 2.7925
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

For 4th trial we chose these settings
epochs = 3
learning_rate = .005
input_size = 20
Neurons = 50
activation_input = 50
activation function = relu
output = 1

In [281]:
epochs = 3
learning_rate = .005


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net4 = torch.nn.Sequential(torch.nn.Linear(20,50),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(50,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net4.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net4.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net4[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-281-91f5dcc44d66>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 0.2787
Epoch 3, Sample 11007: Loss: 0.3455
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.2630
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.2728
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 1.4884
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 0.0968
Epoch 3, Sample 11019: Loss: 0.1207
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

For 5th trial we chose these settings
epochs = 3
learning_rate = .005
input_size = 20
Neurons = 30
activation_input = 30
activation function = relu
output = 1

In [282]:
epochs = 3
learning_rate = .005


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net5 = torch.nn.Sequential(torch.nn.Linear(20,30),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(30,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net5.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net5.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net5[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-282-badd8fb447d0>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 0.0009
Epoch 3, Sample 11007: Loss: 0.0003
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.0864
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.4118
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 0.1733
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 1.0228
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

For 6th trial we chose these settings
epochs = 3
learning_rate = .005
input_size = 20
Neurons = 70
activation_input = 70
activation function = relu
output = 1

In [283]:
epochs = 3
learning_rate = .005


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net6 = torch.nn.Sequential(torch.nn.Linear(20,70),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(70,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net6.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net6.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net6[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-283-6ab5608572ae>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 0.0213
Epoch 3, Sample 11007: Loss: 0.0031
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.2831
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.4243
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 0.1677
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 1.4611
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

For 7th trial we chose these settings
epochs = 3
learning_rate = .005
input_size = 20
Neurons = 150
activation_input = 150
activation function = relu
output = 1

In [284]:
epochs = 3
learning_rate = .005


# Initializing Parameters
w = 50

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net7 = torch.nn.Sequential(torch.nn.Linear(20,150),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(150,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net7.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net7.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net7[0].weight.data[0][0]))

      # Display the loss for the current sample
      print(f"Epoch {epoch+1}, Sample {i+1}: Loss: {loss.item():.4f}")


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-284-763b4addd60f>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Epoch 3, Sample 11005: Loss: 0.2832
Epoch 3, Sample 11006: Loss: 0.1296
Epoch 3, Sample 11007: Loss: 0.0322
Epoch 3, Sample 11008: Loss: 0.7036
Epoch 3, Sample 11009: Loss: 0.5125
Epoch 3, Sample 11010: Loss: 0.0008
Epoch 3, Sample 11011: Loss: 0.0026
Epoch 3, Sample 11012: Loss: 0.7453
Epoch 3, Sample 11013: Loss: 0.1437
Epoch 3, Sample 11014: Loss: 0.2032
Epoch 3, Sample 11015: Loss: 1.2107
Epoch 3, Sample 11016: Loss: 0.3399
Epoch 3, Sample 11017: Loss: 0.2339
Epoch 3, Sample 11018: Loss: 1.9680
Epoch 3, Sample 11019: Loss: 0.0505
Epoch 3, Sample 11020: Loss: 0.1170
Epoch 3, Sample 11021: Loss: 0.4663
Epoch 3, Sample 11022: Loss: 1.6418
Epoch 3, Sample 11023: Loss: 0.2750
Epoch 3, Sample 11024: Loss: 0.9367
Epoch 3, Sample 11025: Loss: 0.0025
Epoch 3, Sample 11026: Loss: 1.1265
Epoch 3, Sample 11027: Loss: 1.0155
Epoch 3, Sample 11028: Loss: 0.7036
Epoch 3, Sample 11029: Loss: 0.5871
Epoch 3, Sample 11030: Loss: 0.0212
Epoch 3, Samp

We could see that the sixth iteration had the lowest average loss, so we decided to use that one for our test validation

In [285]:
# Convert X_test to PyTorch tensor
tensor_test_X = torch.tensor(X_test, dtype=torch.float32)

# Convert y_test to PyTorch tensor
tensor_test_Y = torch.tensor(y_test, dtype=torch.float32)


In [286]:
# Now lets save our trained model
# save the trained model
torch.save(model_net6, 'model_net6.pkl')

In [287]:
#   Lets load the model for inference

model_net6_trained = torch.load('model_net6.pkl')
model_net6_trained.eval()

Sequential(
  (0): Linear(in_features=20, out_features=70, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.33, inplace=False)
  (3): Linear(in_features=70, out_features=1, bias=True)
  (4): ReLU()
)

In [288]:
# Print the parameters of all layers
for name, param in model_net6_trained.named_parameters():
    print(f"Layer: {name}")
    print(f"Size: {param.size()}")
    print(f"Values: \n{param.data}\n")

Layer: 0.weight
Size: torch.Size([70, 20])
Values: 
tensor([[-0.3908,  0.1770,  0.1061,  ...,  0.1203, -0.1229,  0.1744],
        [ 0.0890, -0.5378,  0.0921,  ..., -0.0056, -0.0719,  0.0807],
        [ 0.0198,  0.3877, -0.0649,  ..., -0.0637,  0.0533,  0.2519],
        ...,
        [ 0.0132,  0.7884, -0.0078,  ...,  0.0720,  0.0394,  0.0225],
        [ 0.0857,  0.5062,  0.1818,  ...,  0.0779, -0.0653,  0.0943],
        [ 0.0963,  0.2689, -0.0769,  ...,  0.1805, -0.1895, -0.0557]])

Layer: 0.bias
Size: torch.Size([70])
Values: 
tensor([-1.4038e-01, -4.3543e-01, -1.3614e-01, -2.3436e-01,  3.2598e-02,
        -3.8732e-01, -4.0747e-01,  2.0155e-02, -4.8901e-01, -5.1958e-01,
        -2.3408e-01, -2.1652e-01, -4.7086e-01, -6.7362e-01,  1.5375e-01,
         1.7421e-01, -9.2186e-02,  2.5974e-01, -1.3835e-01,  3.3796e-01,
        -2.0318e-01, -2.0343e-01, -5.7636e-01,  4.3888e-01,  1.5453e-01,
         1.0880e-01,  1.9838e-01,  1.3791e-01,  1.8012e-02,  1.2548e-01,
         5.1607e-01, -2.0668e

In [292]:
# Forward Pass
with torch.no_grad():
    model_net6.eval()  # Set the model to evaluation mode
    predictions = model_net6(tensor_test_X)

# Print the predictions
print(predictions)

tensor([[1.8196],
        [1.2584],
        [0.0000],
        ...,
        [1.4436],
        [0.0000],
        [0.0000]])


In [294]:

test_loss = 0.0

# Loop over each sample in the test dataset
for i in range(tensor_test_X.size(0)):

    # 1. Forward Pass
    output = model_net6.forward(tensor_test_X[i].reshape(-1))

    # 2. Calculate Loss
    loss = loss_mse(output, tensor_test_Y[i].reshape(-1))

    # Aggregate test loss
    test_loss += loss.item()

    # Display the loss for the current sample
    print(f"Sample {i+1}: Test Loss: {loss.item():.4f}")

# Calculate and display average test loss
test_loss /= tensor_test_X.size(0)
print(f"\nAverage Test Loss: {test_loss:.4f}")


Sample 1: Test Loss: 0.5792
Sample 2: Test Loss: 0.3916
Sample 3: Test Loss: 0.0630
Sample 4: Test Loss: 0.0026
Sample 5: Test Loss: 0.2879
Sample 6: Test Loss: 0.0300
Sample 7: Test Loss: 0.0205
Sample 8: Test Loss: 0.7433
Sample 9: Test Loss: 0.3441
Sample 10: Test Loss: 0.8951
Sample 11: Test Loss: 0.0000
Sample 12: Test Loss: 0.2339
Sample 13: Test Loss: 0.7036
Sample 14: Test Loss: 0.3399
Sample 15: Test Loss: 0.2032
Sample 16: Test Loss: 0.0234
Sample 17: Test Loss: 0.2032
Sample 18: Test Loss: 0.2036
Sample 19: Test Loss: 0.1559
Sample 20: Test Loss: 0.0000
Sample 21: Test Loss: 0.0838
Sample 22: Test Loss: 1.9666
Sample 23: Test Loss: 0.0590
Sample 24: Test Loss: 0.3441
Sample 25: Test Loss: 0.0018
Sample 26: Test Loss: 0.1355
Sample 27: Test Loss: 0.3399
Sample 28: Test Loss: 0.0610
Sample 29: Test Loss: 0.7036
Sample 30: Test Loss: 0.5196
Sample 31: Test Loss: 0.0364
Sample 32: Test Loss: 0.2759
Sample 33: Test Loss: 0.7036
Sample 34: Test Loss: 0.5550
Sample 35: Test Loss: 4